# Question 1: Data Structure and Processing Pipeline (15 marks)

In [5]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Load the iris dataset

a) Create a data processing class that implements:


● Conversion of data to pandas DataFrame with proper column names


● Feature scaling using StandardScaler


● Train-test split with experiment tracking

In [6]:
from sklearn.preprocessing import StandardScaler
class IrisDataProcessor:
    def __init__(self):
        # Initialize your experiment
        self.data = load_iris()
        self.df = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None

    def prepare_data(self):
        # Implement experiment workflow
        self.df = pd.DataFrame(self.data.data, columns=self.data.feature_names)
        self.df['target'] = self.data.target
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(self.df[self.data.feature_names])
        self.df[self.data.feature_names] = features_scaled
        
        X = self.df[self.data.feature_names]        # Train-test split
        y = self.df['target']
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        print("Processed Dataset (first 5 rows):\n", self.df.head())
        print("Train-test split shapes:", self.X_train.shape, self.X_test.shape)

    def get_feature_stats(self):
        stats = self.df.describe()
        print("Feature statistics:\n", stats)

# Instantiate and run
processor = IrisDataProcessor()
processor.prepare_data()
processor.get_feature_stats()

Processed Dataset (first 5 rows):
    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0          -0.900681          1.019004          -1.340227         -1.315444   
1          -1.143017         -0.131979          -1.340227         -1.315444   
2          -1.385353          0.328414          -1.397064         -1.315444   
3          -1.506521          0.098217          -1.283389         -1.315444   
4          -1.021849          1.249201          -1.340227         -1.315444   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
Train-test split shapes: (120, 4) (30, 4)
Feature statistics:
        sepal length (cm)  sepal width (cm)  petal length (cm)  \
count       1.500000e+02      1.500000e+02       1.500000e+02   
mean       -1.468455e-15     -1.823726e-15      -1.610564e-15   
std         1.003350e+00      1.003350e+00       1.003350e+00   
min        -1.870024e+00     -2.433947e+00      -1.567576e+00   
25%        -9.006812e-01     -